<a href="https://colab.research.google.com/github/seonghaiiiii/jeju-food/blob/main/zero_yesterday_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.metrics import mean_squared_error, r2_score

#Data load


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# data load
train = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/train.csv')
trade = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/international_trade.csv')
test = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/test.csv')

#train 기본 정보

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59397 entries, 0 to 59396
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           59397 non-null  object 
 1   timestamp    59397 non-null  object 
 2   item         59397 non-null  object 
 3   corporation  59397 non-null  object 
 4   location     59397 non-null  object 
 5   supply(kg)   59397 non-null  float64
 6   price(원/kg)  59397 non-null  float64
dtypes: float64(2), object(5)
memory usage: 3.2+ MB


In [ ]:
train.head()

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


#데이터 자르기

In [ ]:
# ID 열의 앞 6자를 기준으로 groupby하여 데이터프레임을 분할
grouped = train.groupby(train['ID'].str[:6])

# 그룹별로 데이터프레임을 만들어 저장
dataframes = {}
for group_name, group_data in grouped:
    dataframes[group_name] = group_data

# 결과 확인
for group_name, group_df in dataframes.items():
    print(f"Group: {group_name}")
    print(group_df)


Group: BC_A_J
                    ID   timestamp item corporation location  supply(kg)  \
42644  BC_A_J_20190101  2019-01-01   BC           A        J         0.0   
42645  BC_A_J_20190102  2019-01-02   BC           A        J         0.0   
42646  BC_A_J_20190103  2019-01-03   BC           A        J      7616.0   
42647  BC_A_J_20190104  2019-01-04   BC           A        J      7488.0   
42648  BC_A_J_20190105  2019-01-05   BC           A        J     10408.0   
...                ...         ...  ...         ...      ...         ...   
44162  BC_A_J_20230227  2023-02-27   BC           A        J     11702.0   
44163  BC_A_J_20230228  2023-02-28   BC           A        J      6056.0   
44164  BC_A_J_20230301  2023-03-01   BC           A        J      5896.0   
44165  BC_A_J_20230302  2023-03-02   BC           A        J      8600.0   
44166  BC_A_J_20230303  2023-03-03   BC           A        J     10728.0   

       price(원/kg)  
42644          0.0  
42645          0.0  
42646     

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/sample_submission.csv')

In [ ]:
# 'ID' 열의 고유한 값 확인
unique_ids = df1['ID'].unique()

# 고유한 값들 출력
for unique_id in unique_ids:
    print(unique_id)

TG_A_J_20230304
TG_A_J_20230305
TG_A_J_20230306
TG_A_J_20230307
TG_A_J_20230308
TG_A_J_20230309
TG_A_J_20230310
TG_A_J_20230311
TG_A_J_20230312
TG_A_J_20230313
TG_A_J_20230314
TG_A_J_20230315
TG_A_J_20230316
TG_A_J_20230317
TG_A_J_20230318
TG_A_J_20230319
TG_A_J_20230320
TG_A_J_20230321
TG_A_J_20230322
TG_A_J_20230323
TG_A_J_20230324
TG_A_J_20230325
TG_A_J_20230326
TG_A_J_20230327
TG_A_J_20230328
TG_A_J_20230329
TG_A_J_20230330
TG_A_J_20230331
TG_A_S_20230304
TG_A_S_20230305
TG_A_S_20230306
TG_A_S_20230307
TG_A_S_20230308
TG_A_S_20230309
TG_A_S_20230310
TG_A_S_20230311
TG_A_S_20230312
TG_A_S_20230313
TG_A_S_20230314
TG_A_S_20230315
TG_A_S_20230316
TG_A_S_20230317
TG_A_S_20230318
TG_A_S_20230319
TG_A_S_20230320
TG_A_S_20230321
TG_A_S_20230322
TG_A_S_20230323
TG_A_S_20230324
TG_A_S_20230325
TG_A_S_20230326
TG_A_S_20230327
TG_A_S_20230328
TG_A_S_20230329
TG_A_S_20230330
TG_A_S_20230331
TG_B_J_20230304
TG_B_J_20230305
TG_B_J_20230306
TG_B_J_20230307
TG_B_J_20230308
TG_B_J_20230309
TG_B_J_2

In [ ]:
#ID 앞에 6자만 필요해서 자른거임
df1['ID_1'] = df1['ID'].str[:6]

In [ ]:
# 'ID' 열의 고유한 값 확인
unique_ids = df1['ID_1'].unique()

unique_ids

# 총 39개

array(['TG_A_J', 'TG_A_S', 'TG_B_J', 'TG_B_S', 'TG_C_J', 'TG_C_S',
       'TG_D_J', 'TG_D_S', 'TG_E_J', 'TG_E_S', 'CR_A_J', 'CR_B_J',
       'CR_C_J', 'CR_D_J', 'CR_D_S', 'CR_E_J', 'CR_E_S', 'CB_A_J',
       'CB_A_S', 'CB_D_J', 'CB_E_J', 'RD_A_J', 'RD_A_S', 'RD_C_S',
       'RD_D_J', 'RD_D_S', 'RD_E_J', 'RD_E_S', 'BC_A_J', 'BC_A_S',
       'BC_B_J', 'BC_B_S', 'BC_C_J', 'BC_C_S', 'BC_D_J', 'BC_E_J',
       'BC_E_S', 'CB_F_J', 'RD_F_J'], dtype=object)

In [ ]:
TG_A_J = dataframes.get('TG_A_J')
TG_A_S = dataframes.get('TG_A_S')
TG_B_J = dataframes.get('TG_B_J')
TG_B_S = dataframes.get('TG_B_S')
TG_C_J = dataframes.get('TG_C_J')
TG_C_S = dataframes.get('TG_C_S')
TG_D_J = dataframes.get('TG_D_J')
TG_D_S = dataframes.get('TG_D_S')
TG_E_J = dataframes.get('TG_E_J')
TG_E_S = dataframes.get('TG_E_S')
CR_A_J = dataframes.get('CR_A_J')
CR_B_J = dataframes.get('CR_B_J')
CR_C_J = dataframes.get('CR_C_J')
CR_D_J = dataframes.get('CR_D_J')
CR_D_S = dataframes.get('CR_D_S')
CR_E_J = dataframes.get('CR_E_J')
CR_E_S = dataframes.get('CR_E_S')
CB_A_J = dataframes.get('CB_A_J')
CB_A_S = dataframes.get('CB_A_S')
CB_D_J = dataframes.get('CB_D_J')
CB_E_J = dataframes.get('CB_E_J')
RD_A_J = dataframes.get('RD_A_J')
RD_A_S = dataframes.get('RD_A_S')
RD_C_S = dataframes.get('RD_C_S')
RD_D_J = dataframes.get('RD_D_J')
RD_D_S = dataframes.get('RD_D_S')
RD_E_J = dataframes.get('RD_E_J')
RD_E_S = dataframes.get('RD_E_S')
BC_A_J = dataframes.get('BC_A_J')
BC_A_S = dataframes.get('BC_A_S')
BC_B_J = dataframes.get('BC_B_J')
BC_B_S = dataframes.get('BC_B_S')
BC_C_J = dataframes.get('BC_C_J')
BC_C_S = dataframes.get('BC_C_S')
BC_D_J = dataframes.get('BC_D_J')
BC_E_J = dataframes.get('BC_E_J')
BC_E_S = dataframes.get('BC_E_S')
CB_F_J = dataframes.get('CB_F_J')
RD_F_J = dataframes.get('RD_F_J')

In [ ]:
# 삭제할 열 이름 리스트
columns_to_delete = ['item', 'corporation', 'location','supply(kg)']
# 변수들에 대해 열 삭제 수행
dataframes = [TG_A_J, TG_A_S, TG_B_J, TG_B_S, TG_C_J, TG_C_S, TG_D_J, TG_D_S, TG_E_J, TG_E_S,
              CR_A_J, CR_B_J, CR_C_J, CR_D_J, CR_D_S, CR_E_J, CR_E_S,
              CB_A_J, CB_A_S, CB_D_J, CB_E_J,
              RD_A_J, RD_A_S, RD_C_S, RD_D_J, RD_D_S, RD_E_J, RD_E_S,
              BC_A_J, BC_A_S, BC_B_J, BC_B_S, BC_C_J, BC_C_S, BC_D_J, BC_E_J, BC_E_S,
              CB_F_J, RD_F_J]

for df in dataframes:
    df.drop(columns=columns_to_delete, inplace=True)


In [ ]:
TG_A_J.head(10)

,ID,timestamp,price(원/kg)
0,TG_A_J_20190101,2019-01-01,0.0
1,TG_A_J_20190102,2019-01-02,0.0
2,TG_A_J_20190103,2019-01-03,1728.0
3,TG_A_J_20190104,2019-01-04,1408.0
4,TG_A_J_20190105,2019-01-05,1250.0
5,TG_A_J_20190106,2019-01-06,0.0
6,TG_A_J_20190107,2019-01-07,1474.0
7,TG_A_J_20190108,2019-01-08,1326.0
8,TG_A_J_20190109,2019-01-09,1428.0
9,TG_A_J_20190110,2019-01-10,1433.0


In [ ]:
CR_A_J.head(11)

,timestamp,price(원/kg)
15230,2019-01-01,0.0
15231,2019-01-02,0.0
15232,2019-01-03,0.0
15233,2019-01-04,1141.0
15234,2019-01-05,1133.0
15235,2019-01-06,0.0
15236,2019-01-07,0.0
15237,2019-01-08,0.0
15238,2019-01-09,1112.0
15239,2019-01-10,0.0


In [ ]:
TG_A_J.head(10)

,timestamp,price(원/kg)
0,2019-01-01,0.0
1,2019-01-02,0.0
2,2019-01-03,1728.0
3,2019-01-04,1408.0
4,2019-01-05,1250.0
5,2019-01-06,0.0
6,2019-01-07,1474.0
7,2019-01-08,1326.0
8,2019-01-09,1428.0
9,2019-01-10,1433.0


In [ ]:
CR_A_J.head(11)

,timestamp,price(원/kg)
15230,2019-01-01,0.0
15231,2019-01-02,0.0
15232,2019-01-03,0.0
15233,2019-01-04,1141.0
15234,2019-01-05,1133.0
15235,2019-01-06,0.0
15236,2019-01-07,0.0
15237,2019-01-08,0.0
15238,2019-01-09,1112.0
15239,2019-01-10,0.0


In [ ]:
train.iloc[1:2]

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0


In [ ]:
train.iloc[2:3]

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0


In [ ]:
train.iloc[1:2].append(train.iloc[2:3])

<ipython-input-19-dedf0b4a0551>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train.iloc[1:2].append(train.iloc[2:3])


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0


In [ ]:
CR_A_J.head(11)

,timestamp,price(원/kg)
15230,2019-01-01,0.0
15231,2019-01-02,0.0
15232,2019-01-03,0.0
15233,2019-01-04,1141.0
15234,2019-01-05,1133.0
15235,2019-01-06,0.0
15236,2019-01-07,0.0
15237,2019-01-08,0.0
15238,2019-01-09,1112.0
15239,2019-01-10,0.0


In [ ]:
zero_price_indices = CR_A_J[CR_A_J['price(원/kg)'] == 0].index
zero_price_indices

Int64Index([15230, 15231, 15232, 15235, 15236, 15237, 15239, 15241, 15242,
            15244,
            ...
            16712, 16713, 16714, 16719, 16725, 16726, 16733, 16739, 16740,
            16747],
           dtype='int64', length=1003)

In [ ]:
# 인덱스 뒤집기
reversed_indices = zero_price_indices[::-1]

# 뒤집은 인덱스 출력
reversed_indices


Int64Index([16747, 16740, 16739, 16733, 16726, 16725, 16719, 16714, 16713,
            16712,
            ...
            15244, 15242, 15241, 15239, 15237, 15236, 15235, 15232, 15231,
            15230],
           dtype='int64', length=1003)

In [ ]:
# 0인 행들을 찾아서 바로 전 행의 값으로 대체하는 코드

dataframes = [TG_A_J, TG_A_S, TG_B_J, TG_B_S, TG_C_J, TG_C_S, TG_D_J, TG_D_S, TG_E_J, TG_E_S]


for df in dataframes:
    zero_price_indices = df[df['price(원/kg)'] == 0].index
    reversed_indices = zero_price_indices[::-1]
    for index in reversed_indices:
        # Ensure not to go out of bounds
        if index > 0:
            # Replace the zero value with the value of the previous row if the index exists
            if (index - 1) in df.index:
                df.at[index, 'price(원/kg)'] = df.at[index - 1, 'price(원/kg)']

In [ ]:
CR_B_J.head(11)

,ID,timestamp,price(원/kg)
16753,CR_B_J_20190101,2019-01-01,0.0
16754,CR_B_J_20190102,2019-01-02,0.0
16755,CR_B_J_20190103,2019-01-03,1177.0
16756,CR_B_J_20190104,2019-01-04,1177.0
16757,CR_B_J_20190105,2019-01-05,0.0
16758,CR_B_J_20190106,2019-01-06,0.0
16759,CR_B_J_20190107,2019-01-07,1196.0
16760,CR_B_J_20190108,2019-01-08,1196.0
16761,CR_B_J_20190109,2019-01-09,1187.0
16762,CR_B_J_20190110,2019-01-10,1011.0


In [ ]:
train

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [ ]:
group_names = [TG_A_J, TG_A_S, TG_B_J, TG_B_S, TG_C_J, TG_C_S, TG_D_J, TG_D_S, TG_E_J, TG_E_S,
               CR_A_J, CR_B_J, CR_C_J, CR_D_J, CR_D_S, CR_E_J, CR_E_S,
               CB_A_J, CB_A_S, CB_D_J, CB_E_J,
               RD_A_J, RD_A_S, RD_C_S, RD_D_J, RD_D_S, RD_E_J, RD_E_S,
               BC_A_J, BC_A_S, BC_B_J, BC_B_S, BC_C_J, BC_C_S, BC_D_J, BC_E_J, BC_E_S,
               CB_F_J, RD_F_J]
for current_dataframe in group_names:

    # train과 ID가 같으면 train의 price(원/kg)을 각 데이터프레임의 price(원/kg)로 바꾸기
    train = pd.merge(train, current_dataframe[['ID', 'price(원/kg)']], on='ID', how='left', suffixes=('_train', f'_{current_dataframe}'))
    train[f'price(원/kg)_train'] = train[f'price(원/kg)_{current_dataframe}'].combine_first(train['price(원/kg)_train'])
    train.drop(f'price(원/kg)_{current_dataframe}', axis=1, inplace=True)
    train.rename(columns={'price(원/kg)_train': 'price(원/kg)'}, inplace=True)

# 최종 결과 확인
train.head(10)

KeyError: ignored

In [ ]:
train

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [ ]:
train_df = train[['ID','timestamp','supply(kg)', 'price(원/kg)']]
# test_df = pd.read_csv('test.csv')[['ID','timestamp']]

In [ ]:
train_df['item_id'] = train_df.ID.str[0:6]
# test_df['item_id'] = test_df.ID.str[0:6]

<ipython-input-17-a9e14e50f493>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['item_id'] = train_df.ID.str[0:6]


In [ ]:
!pip install -U autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.9 M

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
data = TimeSeriesDataFrame(train_df.drop(columns=['ID']))
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",
)
# seed 고정
predictor.fit( data, random_seed=42, )

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231114_064207/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(원/kg)'
	past covariates:  ['supply(kg)']


In [ ]:
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	10.64   s     = Training runtime
Fitting model: DeepAR_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'Theta_FULL', 'AutoETS_FULL', 'RecursiveTabular_FULL', 'DeepAR_FULL', 'WeightedEnsemble_FULL']
Total runtime: 10.79 s
Updated best model to 'WeightedEnsemble_FULL' (Previously '

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR': 'DeepAR_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [ ]:
# seed 고정
pred = predictor.predict(data, random_seed=42, )

INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [ ]:
pred

mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3521.401908  1992.975442  2557.999164  2940.728362   
        2023-03-05  3446.539062  1313.569420  2102.010286  2584.000447   
        2023-03-06  3305.159501   753.464331  1626.564936  2260.689727   
        2023-03-07  3545.757786   689.616279  1659.639798  2368.194996   
        2023-03-08  3436.158184   184.994242  1339.792564  2099.346175   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   510.054373    45.669588   231.000420   342.538382   
        2023-03-28   512.209125    52.800895   224.875815   341.706775   
        2023-03-29   517.048563    52.109742   222.724186   336.587660   
        2023-03-30   503.933168    44.637221   212.060385   327.526812   
        2023-03-31   502.447305    47.392048   205.522628   320.476887   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3245.918945  3522.824760  3811.098372  4129.684463   
        2023-03-05  3024.640596  3432.986328  3823.985990  4283.033366   
        2023-03-06  2814.100256  3308.919510  3814.025874  4334.655275   
        2023-03-07  2991.677481  3549.857884  4086.111040  4684.275776   
        2023-03-08  2770.465102  3393.690777  3986.482004  4694.239603   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   438.746630   519.355795   601.088176   691.565971   
        2023-03-28   433.174680   517.907489   606.099893   696.546543   
        2023-03-29   433.436336   516.603342   599.801903   694.996343   
        2023-03-30   419.325362   515.442156   601.839787   690.394777   
        2023-03-31   420.189931   505.257348   593.043638   693.306395   

                            0.8          0.9  
item_id timestamp                             
TG_A_J  2023-03-04  4524.784633  5053.775884  
        2023-03-05  4791.366178  5598.791175  
        2023-03-06  4954.837928  5820.807673  
        2023-03-07  5387.344891  6403.313843  
        2023-03-08  5493.383546  6578.203963  
...                         ...          ...  
RD_F_J  2023-03-27   800.892964   955.219234  
        2023-03-28   801.607696   954.045844  
        2023-03-29   809.038849   969.386707  
        2023-03-30   801.129361   964.450963  
        2023-03-31   801.413730   961.710722  

[1092 rows x 10 columns]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/sample_submission.csv')

In [ ]:
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
submission

,ID,answer
0,TG_A_J_20230304,3521.401908
1,TG_A_J_20230305,3446.539062
2,TG_A_J_20230306,3305.159501
3,TG_A_J_20230307,3545.757786
4,TG_A_J_20230308,3436.158184
...,...,...
1087,RD_F_J_20230327,510.054373
1088,RD_F_J_20230328,512.209125
1089,RD_F_J_20230329,517.048563
1090,RD_F_J_20230330,503.933168


In [ ]:
from datetime import datetime

# 'ID' 열에서 뒤 8글자를 추출하여 'date' 열을 만듭니다.
submission['date'] = pd.to_datetime(submission['ID'].str[-8:], format='%Y%m%d')

# 'date' 열에서 요일을 추출하여 'day_of_week' 열을 만듭니다. (0: 월요일, 1: 화요일, ..., 6: 일요일)
submission['day_of_week'] = submission['date'].dt.dayofweek

# 'day_of_week'이 6인 (일요일) 행들의 'answer' 값을 0으로 설정합니다.
submission.loc[submission['day_of_week'] == 6, 'answer'] = 0

# 'date' 및 'day_of_week' 열은 더 이상 필요하지 않으므로 제거합니다.
submission = submission.drop(['date', 'day_of_week'], axis=1)


In [ ]:
submission

,ID,answer
0,TG_A_J_20230304,3521.401908
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3305.159501
3,TG_A_J_20230307,3545.757786
4,TG_A_J_20230308,3436.158184
...,...,...
1087,RD_F_J_20230327,510.054373
1088,RD_F_J_20230328,512.209125
1089,RD_F_J_20230329,517.048563
1090,RD_F_J_20230330,503.933168


In [ ]:
submission.to_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/부계정제출.csv', index=False)

In [ ]:
CR_A_J.head(30)

,timestamp,price(원/kg)
15230,2019-01-01,0.0
15231,2019-01-02,0.0
15232,2019-01-03,0.0
15233,2019-01-04,1141.0
15234,2019-01-05,1133.0
15235,2019-01-06,0.0
15236,2019-01-07,0.0
15237,2019-01-08,0.0
15238,2019-01-09,1112.0
15239,2019-01-10,0.0


In [ ]:
# 변수들에 대해 열 이름 변경 수행
dataframes = [TG_A_J, TG_A_S, TG_B_J, TG_B_S, TG_C_J, TG_C_S, TG_D_J, TG_D_S, TG_E_J, TG_E_S,
              CR_A_J, CR_B_J, CR_C_J, CR_D_J, CR_D_S, CR_E_J, CR_E_S,
              CB_A_J, CB_A_S, CB_D_J, CB_E_J,
              RD_A_J, RD_A_S, RD_C_S, RD_D_J, RD_D_S, RD_E_J, RD_E_S,
              BC_A_J, BC_A_S, BC_B_J, BC_B_S, BC_C_J, BC_C_S, BC_D_J, BC_E_J, BC_E_S,
              CB_F_J, RD_F_J]

for df in dataframes:
    if 'price(원/kg)' in df.columns:
        df.rename(columns={'price': 'price(원/kg'}, inplace=True)

In [ ]:
CR_E_S

,timestamp,price
24368,2019-01-01,0.0
24369,2019-01-02,0.0
24370,2019-01-03,0.0
24371,2019-01-04,0.0
24372,2019-01-05,0.0
...,...,...
25886,2023-02-27,0.0
25887,2023-02-28,0.0
25888,2023-03-01,0.0
25889,2023-03-02,0.0


In [ ]:
TG_A_J.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1523 entries, 0 to 1522
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  1523 non-null   object 
 1   price      1523 non-null   float64
dtypes: float64(1), object(1)
memory usage: 68.0+ KB


In [ ]:
TG_A_J.describe()

,price
count,1523.000000
mean,4262.806303
std,2858.382464
min,0.000000
25%,1567.500000
50%,3686.000000
75%,6798.250000
max,11704.000000


In [ ]:
TG_A_S.head(10)

,ID,timestamp,price
1523,TG_A_S_20190101,2019-01-01,0.0
1524,TG_A_S_20190102,2019-01-02,0.0
1525,TG_A_S_20190103,2019-01-03,2526.0
1526,TG_A_S_20190104,2019-01-04,2134.0
1527,TG_A_S_20190105,2019-01-05,2075.0
1528,TG_A_S_20190106,2019-01-06,2075.0
1529,TG_A_S_20190107,2019-01-07,2181.0
1530,TG_A_S_20190108,2019-01-08,2029.0
1531,TG_A_S_20190109,2019-01-09,1998.0
1532,TG_A_S_20190110,2019-01-10,2023.0


#아마 이게 최종? price lstm 예측

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from datetime import datetime, timedelta

# Function to prepare the dataset for LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

# List of dataframes
dataframes = [TG_A_J, TG_A_S, TG_B_J, TG_B_S, TG_C_J, TG_C_S, TG_D_J, TG_D_S, TG_E_J, TG_E_S,
              CR_A_J, CR_B_J, CR_C_J, CR_D_J, CR_D_S, CR_E_J, CR_E_S,
              CB_A_J, CB_A_S, CB_D_J, CB_E_J,
              RD_A_J, RD_A_S, RD_C_S, RD_D_J, RD_D_S, RD_E_J, RD_E_S,
              BC_A_J, BC_A_S, BC_B_J, BC_B_S, BC_C_J, BC_C_S, BC_D_J, BC_E_J, BC_E_S,
              CB_F_J, RD_F_J]

# Initialize a dictionary to store predicted DataFrames
predicted_dfs = {}

# Loop through each dataframe
for i, df in enumerate(dataframes):
    # Convert timestamp to datetime and add a new column
    df['datetime'] = pd.to_datetime(df['timestamp'])

    # Extract relevant columns
    data = df[['datetime', 'price']]

    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    data[['price']] = scaler.fit_transform(data[['price']])

    # Set the look-back period (you can adjust this)
    look_back = 1500

    # Create the dataset
    X, Y = create_dataset(data[['price']].values, look_back)

    # Reshape input to be [samples, time steps, features]
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))

    # Build the LSTM model with early stopping
    model = Sequential()
    model.add(LSTM(units=50, input_shape=(1, look_back), return_sequences=True))  # 첫 번째 LSTM 층
    model.add(LSTM(units=50, return_sequences=True))  # 두 번째 LSTM 층
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')


    # Set up early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Fit the model with early stopping
    model.fit(X, Y, epochs=100, batch_size=1, verbose=2, callbacks=[early_stopping])

    # Generate future timestamps from 2023-03-04 to 2023-03-31
    future_dates = pd.date_range(start='2023-03-04', end='2023-03-31')


    # 나머지 코드는 동일하게 유지

    # ...[이전 코드]...

    # Make predictions for the future dates
    predicted_prices_normalized = []

    for j in range(len(future_dates)):
        # 마지막 look_back 개수의 가격 데이터를 선택
        last_prices = data['price'][-look_back:].values
        # 데이터 형태를 모델의 입력 형태에 맞게 조정
        input_data = np.reshape(last_prices, (1, 1, look_back))
        # 데이터 타입을 float32로 변환
        input_data = input_data.astype('float32')

        # 모델 예측
        predicted_price_normalized = model.predict(input_data)[0, 0]
        predicted_prices_normalized.append(predicted_price_normalized)
        # 예측된 가격을 데이터에 추가
        data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)

    # ...[이후 코드]...


    # Inverse transform the predicted prices to get the actual values
    predicted_prices = scaler.inverse_transform(np.array(predicted_prices_normalized).reshape(-1, 1))

    # Create a DataFrame for the predicted prices and corresponding timestamps
    predicted_df = pd.DataFrame(index=future_dates, data={'predicted_price': predicted_prices.flatten()})

    # Rename the first column in predicted_df to 'timestamp'
    predicted_df = predicted_df.rename(columns={'predicted_price': 'price'})

    # Add the 'timestamp' column to predicted_df
    predicted_df['timestamp'] = predicted_df.index

    # Store the result in the dictionary
    predicted_dfs[f'predicted_df{i+1}'] = predicted_df

# Display the first few rows of the first predicted dataframe
print(predicted_dfs['predicted_df1'].head())


<ipython-input-23-c6585b621fdf>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[['price']] = scaler.fit_transform(data[['price']])


Epoch 1/100


23/23 - 6s - loss: 0.0080 - 6s/epoch - 251ms/step
Epoch 2/100


23/23 - 0s - loss: 0.0021 - 263ms/epoch - 11ms/step
Epoch 3/100


23/23 - 0s - loss: 7.9872e-04 - 265ms/epoch - 12ms/step
Epoch 4/100


...

<ipython-input-23-c6585b621fdf>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-23-c6585b621fdf>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


1/1 [==============================] - 0s 39ms/step


<ipython-input-23-c6585b621fdf>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-23-c6585b621fdf>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


1/1 [==============================] - 0s 41ms/step
                  price  timestamp
2023-03-04  3139.726318 2023-03-04
2023-03-05  3035.303711 2023-03-05
2023-03-06  2976.633057 2023-03-06
2023-03-07  2926.635498 2023-03-07
2023-03-08  3067.561279 2023-03-08


<ipython-input-23-c6585b621fdf>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)
<ipython-input-23-c6585b621fdf>:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'datetime': future_dates[j], 'price': predicted_price_normalized}, ignore_index=True)


In [ ]:
predicted_dfs['predicted_df1']

,price,timestamp
2023-03-04,3512.169189,2023-03-04
2023-03-05,3300.499756,2023-03-05
2023-03-06,3266.823975,2023-03-06
2023-03-07,3185.840820,2023-03-07
2023-03-08,3239.210693,2023-03-08
2023-03-09,3233.833740,2023-03-09
2023-03-10,3053.960205,2023-03-10
2023-03-11,3347.626221,2023-03-11
2023-03-12,3234.272949,2023-03-12
2023-03-13,2946.496582,2023-03-13


In [ ]:
# Create variables for each predicted dataframe
for i, key in enumerate(predicted_dfs.keys()):
    globals()[f'predicted_df{i+1}'] = predicted_dfs[key]

In [ ]:
predicted_df1

,price,timestamp
2023-03-04,3512.169189,2023-03-04
2023-03-05,3300.499756,2023-03-05
2023-03-06,3266.823975,2023-03-06
2023-03-07,3185.840820,2023-03-07
2023-03-08,3239.210693,2023-03-08
2023-03-09,3233.833740,2023-03-09
2023-03-10,3053.960205,2023-03-10
2023-03-11,3347.626221,2023-03-11
2023-03-12,3234.272949,2023-03-12
2023-03-13,2946.496582,2023-03-13


In [ ]:
# Create variables for each predicted dataframe with new names
new_names = [
    'TG_A_J', 'TG_A_S', 'TG_B_J', 'TG_B_S', 'TG_C_J', 'TG_C_S', 'TG_D_J', 'TG_D_S', 'TG_E_J', 'TG_E_S',
    'CR_A_J', 'CR_B_J', 'CR_C_J', 'CR_D_J', 'CR_D_S', 'CR_E_J', 'CR_E_S',
    'CB_A_J', 'CB_A_S', 'CB_D_J', 'CB_E_J',
    'RD_A_J', 'RD_A_S', 'RD_C_S', 'RD_D_J', 'RD_D_S', 'RD_E_J', 'RD_E_S',
    'BC_A_J', 'BC_A_S', 'BC_B_J', 'BC_B_S', 'BC_C_J', 'BC_C_S', 'BC_D_J', 'BC_E_J', 'BC_E_S',
    'CB_F_J', 'RD_F_J'
]

for i, key in enumerate(predicted_dfs.keys()):
    globals()[f'pred_{new_names[i]}'] = predicted_dfs[key]

In [ ]:
pred_RD_E_J

,price,timestamp
2023-03-04,624.958313,2023-03-04
2023-03-05,290.599640,2023-03-05
2023-03-06,536.498352,2023-03-06
2023-03-07,646.380859,2023-03-07
2023-03-08,463.964325,2023-03-08
2023-03-09,330.467072,2023-03-09
2023-03-10,652.424194,2023-03-10
2023-03-11,657.032471,2023-03-11
2023-03-12,213.052780,2023-03-12
2023-03-13,548.550598,2023-03-13


In [ ]:
import pandas as pd

# Assuming df1 is your DataFrame
# Convert the ID column to string
df1['ID_2'] = df1['ID'].astype(str).str.extract(r'_(\d{8})')

# Convert the extracted date to datetime format
df1['ID_2'] = pd.to_datetime(df1['ID_2'], format='%Y%m%d')

# Display the updated DataFrame
df1

,ID,answer,ID_1,ID_2
0,TG_A_J_20230304,0,TG_A_J,2023-03-04
1,TG_A_J_20230305,0,TG_A_J,2023-03-05
2,TG_A_J_20230306,0,TG_A_J,2023-03-06
3,TG_A_J_20230307,0,TG_A_J,2023-03-07
4,TG_A_J_20230308,0,TG_A_J,2023-03-08
...,...,...,...,...
1087,RD_F_J_20230327,0,RD_F_J,2023-03-27
1088,RD_F_J_20230328,0,RD_F_J,2023-03-28
1089,RD_F_J_20230329,0,RD_F_J,2023-03-29
1090,RD_F_J_20230330,0,RD_F_J,2023-03-30


In [ ]:
dt = [pred_TG_A_J, pred_TG_A_S, pred_TG_B_J, pred_TG_B_S, pred_TG_C_J, pred_TG_C_S, pred_TG_D_J, pred_TG_D_S, pred_TG_E_J, pred_TG_E_S,
    pred_CR_A_J, pred_CR_B_J, pred_CR_C_J, pred_CR_D_J, pred_CR_D_S, pred_CR_E_J, pred_CR_E_S,
    pred_CB_A_J, pred_CB_A_S, pred_CB_D_J, pred_CB_E_J,
    pred_RD_A_J, pred_RD_A_S, pred_RD_C_S, pred_RD_D_J, pred_RD_D_S, pred_RD_E_J, pred_RD_E_S,
    pred_BC_A_J, pred_BC_A_S, pred_BC_B_J, pred_BC_B_S, pred_BC_C_J, pred_BC_C_S, pred_BC_D_J, pred_BC_E_J, pred_BC_E_S,
    pred_CB_F_J, pred_RD_F_J]

new_names = [
    'TG_A_J', 'TG_A_S', 'TG_B_J', 'TG_B_S', 'TG_C_J', 'TG_C_S', 'TG_D_J', 'TG_D_S', 'TG_E_J', 'TG_E_S',
    'CR_A_J', 'CR_B_J', 'CR_C_J', 'CR_D_J', 'CR_D_S', 'CR_E_J', 'CR_E_S',
    'CB_A_J', 'CB_A_S', 'CB_D_J', 'CB_E_J',
    'RD_A_J', 'RD_A_S', 'RD_C_S', 'RD_D_J', 'RD_D_S', 'RD_E_J', 'RD_E_S',
    'BC_A_J', 'BC_A_S', 'BC_B_J', 'BC_B_S', 'BC_C_J', 'BC_C_S', 'BC_D_J', 'BC_E_J', 'BC_E_S',
    'CB_F_J', 'RD_F_J'
]
# Assuming df1 and dt (list of predicted dataframes) are already defined

for i, common_part in enumerate(new_names):
    # Filter df1 to include only rows with the common part in ID_1
    filtered_df1 = df1[df1['ID_1'].str.endswith(common_part)]

    # Merge predicted dataframe and filtered_df1 based on timestamp and ID_2
    merged_df = pd.merge(dt[i], filtered_df1, left_on='timestamp', right_on='ID_2')

    # Update the 'answer' column in df1 with the 'price' column from predicted dataframe
    df1.loc[df1['ID_1'].str.endswith(common_part), 'answer'] = merged_df['price'].values

# Display the updated df1
df1


,ID,answer,ID_1,ID_2
0,TG_A_J_20230304,3139.726318,TG_A_J,2023-03-04
1,TG_A_J_20230305,3035.303711,TG_A_J,2023-03-05
2,TG_A_J_20230306,2976.633057,TG_A_J,2023-03-06
3,TG_A_J_20230307,2926.635498,TG_A_J,2023-03-07
4,TG_A_J_20230308,3067.561279,TG_A_J,2023-03-08
...,...,...,...,...
1087,RD_F_J_20230327,525.482544,RD_F_J,2023-03-27
1088,RD_F_J_20230328,529.977112,RD_F_J,2023-03-28
1089,RD_F_J_20230329,544.453430,RD_F_J,2023-03-29
1090,RD_F_J_20230330,524.178589,RD_F_J,2023-03-30


In [ ]:
pred_RD_E_J

,price,timestamp
2023-03-04,596.166443,2023-03-04
2023-03-05,214.086746,2023-03-05
2023-03-06,549.173340,2023-03-06
2023-03-07,648.362183,2023-03-07
2023-03-08,411.965637,2023-03-08
2023-03-09,245.473785,2023-03-09
2023-03-10,633.703064,2023-03-10
2023-03-11,617.117981,2023-03-11
2023-03-12,163.682648,2023-03-12
2023-03-13,562.568237,2023-03-13


In [ ]:
df1.drop('ID_1', axis=1, inplace=True)

In [ ]:
df1['Day_of_week'] = df1['ID_2'].dt.day_name()

# 'Day_of_week'이 'Sunday'인 행의 '예측 price'를 0으로 설정
df1.loc[df1['Day_of_week'] == 'Sunday', 'answer'] = 0

# 'Day_of_week' 열을 삭제 (필요한 경우)
df1.drop('Day_of_week', axis=1, inplace=True)

In [ ]:
df1.drop('ID_2', axis=1, inplace=True)

In [ ]:
df1

,ID,answer
0,TG_A_J_20230304,3139.726318
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,2976.633057
3,TG_A_J_20230307,2926.635498
4,TG_A_J_20230308,3067.561279
...,...,...
1087,RD_F_J_20230327,525.482544
1088,RD_F_J_20230328,529.977112
1089,RD_F_J_20230329,544.453430
1090,RD_F_J_20230330,524.178589


In [ ]:
df1[df1['answer'] < 0] #나중에 이거 0으로 만들어서 제출하면 점수 대충 1점 더 낮아지는듯

,ID,answer
312,CR_B_J_20230308,-18.314877
320,CR_B_J_20230316,-113.111794
334,CR_B_J_20230330,-27.117477
392,CR_D_S_20230304,-0.264208
394,CR_D_S_20230306,-0.320615
395,CR_D_S_20230307,-0.582921
398,CR_D_S_20230310,-0.680580
399,CR_D_S_20230311,-0.191711
402,CR_D_S_20230314,-0.300994
403,CR_D_S_20230315,-0.815148


In [ ]:
df1.loc[df1['answer'] < 0, 'answer'] = 0

In [ ]:
df1[df1['answer'] < 0]

,ID,answer


In [ ]:
df1.to_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/TG만다듬은거.csv', index = False)

In [ ]:
df1

,ID,answer
0,TG_A_J_20230304,3139.726318
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,2976.633057
3,TG_A_J_20230307,2926.635498
4,TG_A_J_20230308,3067.561279
...,...,...
1087,RD_F_J_20230327,525.482544
1088,RD_F_J_20230328,529.977112
1089,RD_F_J_20230329,544.453430
1090,RD_F_J_20230330,524.178589
